# Import packages and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
POLICY_DATA = '../data/datastorm_policy_data.csv'
AGENT_DATA = '../data/datastorm_agent_data.csv'

In [85]:
df = pd.read_csv(POLICY_DATA, parse_dates=['next_due_dt', 'termination_dt', 
                                           'main_holder_dob', 'spouse_dob', 
                                           'child1_dob', 'child2_dob', 
                                           'child3_dob', 'child4_dob', 
                                           'child5_dob', 
                                           'run_date' ])

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
df = df.drop(columns=['main_holder_occupation_cd', 'product_code'])

In [87]:
df.head()

,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,...,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_name,policy_code
0,20,M,INFORCE,2011/08/21,2019-02-21,NaT,NaN,M,1983-10-01,28,...,NaT,NaN,CASH,NaN,20190131,2020-07-01,C108223,AG108847,HEALTH,POL28535
1,15,M,INFORCE,2006/07/14,2019-09-14,NaT,NaN,F,1958-01-01,48,...,NaT,NaN,CASH,BUSINESSMAN,20190901,2020-07-01,C103349,AG103677,RETIREMENT,POL27771
2,20,Y,INFORCE,2018/12/28,2019-12-28,NaT,NaN,M,1967-11-01,51,...,NaT,NaN,CASH,RECYCLE RACERS,20191101,2020-07-01,C120388,AG103249,INVESTMENT,POL29786
3,15,Q,INFORCE,2018/11/06,2020-02-06,NaT,NaN,M,1989-10-01,29,...,NaT,NaN,CHEQUE,RECEPTIONIST,20191201,2020-07-01,C113155,AG106733,HEALTH,POL24635
4,15,M,LAPSED,2012/04/28,2015-06-28,NaT,OTHERS,M,1988-05-01,24,...,NaT,NaN,CASH,BUSINESSMAN,20190228,2020-07-01,C106155,AG106907,HEALTH,POL7470


# Create Labels

In [146]:
def create_labels(df, train_snap_date, label_last_date):
    snap_df = df[(df.policy_snapshot_as_on == train_snap_date) 
                    & (df.policy_status == 'INFORCE')][['client_code', 
                                                        'policy_code', 
                                                        'product_name']]
    cust_df = snap_df[['client_code']].drop_duplicates().set_index('client_code')
    print(cust_df)
    label_df = df[(df.policy_snapshot_as_on > train_snap_date) 
                     & (df.policy_snapshot_as_on <= label_last_date) 
                     & (df.policy_status == 'INFORCE')][['client_code', 
                                                         'product_name', 
                                                         'policy_code', 
                                                         'policy_snapshot_as_on']]
    
    join_df = label_df.merge(snap_df[['policy_code', 'client_code']], on=['policy_code', 'client_code'], how='left', indicator=True)
    join_df['is_prev'] = join_df._merge.map({'left_only':0, 'both':1}).astype(int)
    
    #customers who can be c
    cross_sell_eligible = join_df.groupby(['client_code', 'policy_snapshot_as_on']).is_prev.sum()
    
    cross_sell_eligible.name = 'cross_sell_eligible'
    cross_sell_eligible = cross_sell_eligible[cross_sell_eligible > 0].reset_index()
    
    joined_cs_eligible = join_df.merge(cross_sell_eligible, on=['client_code', 'policy_snapshot_as_on'], how='left').dropna()
    
    labels = joined_cs_eligible[joined_cs_eligible.is_prev == 0][['client_code', 'product_name']].drop_duplicates()
    labels['value'] = 1
    pivot_labels = labels.pivot_table(values='value', index='client_code', aggfunc='sum', columns='product_name').fillna(0).astype(int)

    return cust_df.merge(pivot_labels, on='client_code', how='left').fillna(0)


# Create Client Dataset

In [212]:
def client_dataset(df, snap_date):
    df_snap = df[(df.policy_snapshot_as_on == snap_date) & (df.policy_status == 'INFORCE')]
    
    customer_specific_features = ['main_holder_gender', 'main_holder_dob',
                              'main_holder_smoker_flag', 'spouse_gender',
                               'spouse_dob', 'spouse_smoker_flag',
                               'child1_dob', 'child1_gender', 'child2_dob',
                               'child2_gender', 'child3_dob', 'child3_gender', 'child4_dob',
                               'child4_gender', 'child5_dob', 'child5_gender',
                               'main_holder_occupation', 
                               'client_code']
    
    df_cust = df_snap[customer_specific_features]

    dob_feat = ['main_holder_dob', 'spouse_dob', 'child1_dob', 'child2_dob',
           'child3_dob', 'child4_dob', 'child5_dob']

    age_feat = []
    for col in dob_feat:
        new_col = col[:-4] + '_age'
        df_cust[new_col] = pd.to_datetime(snap_date, format='%Y%m%d').year - df_cust[col].dt.year
        df_cust = df_cust.drop(columns=col)
        age_feat.append(new_col)

    df_cust_numeric = df_cust.groupby('client_code')[age_feat].median()

    df_cust_cat = df_cust.select_dtypes('object').groupby('client_code').first()

    df_cust_fin = df_cust_cat.join(df_cust_numeric)
    
    return df_cust_fin

In [191]:
snap_date = 20181231

In [203]:
df_snap = df[(df.policy_snapshot_as_on == snap_date) & (df.policy_status == 'INFORCE')]

In [204]:
customer_specific_features = ['main_holder_gender', 'main_holder_dob',
                              'main_holder_smoker_flag', 'spouse_gender',
                               'spouse_dob', 'spouse_smoker_flag',
                               'child1_dob', 'child1_gender', 'child2_dob',
                               'child2_gender', 'child3_dob', 'child3_gender', 'child4_dob',
                               'child4_gender', 'child5_dob', 'child5_gender',
                               'main_holder_occupation', 
                               'client_code']

In [210]:
df_cust = df_snap[customer_specific_features]

dob_feat = ['main_holder_dob', 'spouse_dob', 'child1_dob', 'child2_dob',
       'child3_dob', 'child4_dob', 'child5_dob']

age_feat = []
for col in dob_feat:
    new_col = col[:-4] + '_age'
    df_cust[new_col] = pd.to_datetime(snap_date, format='%Y%m%d').year - df_cust[col].dt.year
    df_cust = df_cust.drop(columns=col)
    age_feat.append(new_col)

df_cust_numeric = df_cust.groupby('client_code')[age_feat].median()

df_cust_cat = df_cust.select_dtypes('object').groupby('client_code').first()

df_cust_fin = df_cust_cat.join(df_cust_numeric)

In [211]:
df_cust_fin

,main_holder_gender,main_holder_smoker_flag,spouse_gender,spouse_smoker_flag,child1_gender,child2_gender,child3_gender,child4_gender,child5_gender,main_holder_occupation,main_holder_age,spouse_age,child1_age,child2_age,child3_age,child4_age,child5_age
client_code,,,,,,,,,,,,,,,,,
C100000,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLOUR & GRAIN MILL W,27.0,NaN,NaN,NaN,NaN,NaN,NaN
C100001,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHENA CULTIVATOR,37.0,NaN,NaN,NaN,NaN,NaN,NaN
C100002,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLOUR & GRAIN MILL W,24.0,NaN,NaN,NaN,NaN,NaN,NaN
C100003,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MINISTER,27.0,NaN,NaN,NaN,NaN,NaN,NaN
C100004,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MINISTER,46.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C154407,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALESMAN,39.0,NaN,NaN,NaN,NaN,NaN,NaN
C154416,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALESMAN,30.0,NaN,NaN,NaN,NaN,NaN,NaN
C154436,F,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
df_cust.select_dtypes('object').groupby('client_code').agg(lambda x: pd.Series.mode(x)[0])

IndexError: index out of bounds

In [8]:
df_cust.groupby()

,main_holder_occupation_cd,main_holder_gender,main_holder_dob,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_smoker_flag,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,main_holder_occupation,client_code
0,249.0,M,1983/10/01,N,F,1989/03/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C108223
1,165.0,F,1958/01/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BUSINESSMAN,C103349
2,330.0,M,1967/11/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RECYCLE RACERS,C120388
3,415.0,M,1989/10/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RECEPTIONIST,C113155
4,165.0,M,1988/05/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BUSINESSMAN,C106155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631417,354.0,M,1981/07/01,N,F,1987/08/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAHOUTS,C109432
631418,165.0,M,1982/11/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BUSINESSMAN,C113916
631419,306.0,M,1958/05/01,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QUARRYMEN,C104807
631420,481.0,F,1992/10/01,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C148592


In [187]:
df.agent_code.nunique()

4665

In [186]:
create_labels(df, 20181231, 20190630)

product_name,EDUCATION,HEALTH,INVESTMENT,PROTECTION,RETIREMENT
client_code,,,,,
C100121,0,1,0,0,0
C100132,0,1,0,0,0
C100133,0,0,1,0,0
C100443,0,0,1,0,0
C100446,0,0,1,0,0
...,...,...,...,...,...
C146490,0,0,1,0,0
C147263,0,0,1,0,0
C147285,0,1,0,0,0
